In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
class Node():
    def __init__(self):
        self.prob = 0
        self.depth = 1
        self.left = None
        self.right = None
        self.rownum = 0
        self.feature = None
        self.threshlod = None
        self.is_terminal = None

In [3]:
class DecisionTreeClassifier():
    def __init__(self,max_depth = 3,min_sample_leaf = 1,min_sample_split = 2):
        self.max_depth = max_depth
        self.min_sample_leaf = min_sample_leaf
        self.min_sample_split = min_sample_split
        self.classes = None
        self.Tree = None
        self.Num = 0
        
    def prob(self,y):
        probs = []
        for class_i in self.classes:
            prob = y[ y==class_i ].shape[0]/y.shape[0]
            probs.append( prob )
        return np.asarray( probs )
    
    def Gini( self,y ):
        return 1 - np.sum((self.prob(y))**2)
    
    def BestFeatureSplit( self,X,y ):
        
        bestSplitFea = None
        bestThre = None
        minGini = 1
        
        for i in range( X.shape[ 1 ] ):
            fea_i = X[ :,i ]
            for term_j in fea_i:
                threshold = term_j
                y_left = y[ fea_i <= threshold ]
                y_right = y[ fea_i > threshold ]
                if y_left.shape[ 0 ]==0 or y_right.shape[ 0 ]==0:
                    continue
                    
                GiniLeft = self.Gini( y_left )
                GiniRight = self.Gini( y_right )
                ConGini = (y_left.shape[ 0 ]*GiniLeft + y_right.shape[0]*GiniRight)/y.shape[ 0 ]
                
                if minGini > ConGini:
                    minGini = ConGini
                    bestSplitFea = i
                    bestThre = threshold
                
        X_col = X[ :,bestSplitFea ]
        X_left ,X_right = X[ X_col <= bestThre,: ],X[ X_col > bestThre,: ]
        y_left, y_right = y[ X_col <= bestThre ],y[ X_col > bestThre]
        
        return bestSplitFea,bestThre,X_left,y_left,X_right,y_right
    
    def CreateTree(self,X,y,node):
        
        
        if node.depth > self.max_depth:
            node.is_terminal = True
            return 
        if X.shape[ 0 ] <= self.min_sample_split:
            node.is_terminal = True
            return 
        if np.unique(y).shape[ 0 ] == 1:
            node.is_terminal = True
            return 
        
        splitCol,thresh,X_left,y_left,X_right,y_right = self.BestFeatureSplit( X,y )
        if X_left.shape[ 0 ] < self.min_sample_leaf or X_right.shape[ 0 ] < self.min_sample_leaf:
            node.is_terminal = True
            return 
        
        node.feature = splitCol
        node.threshlod = thresh
        
        node.left = Node()
        node.left.depth = node.depth +1
        node.left.rownum = y_left.shape[ 0 ]
        node.left.prob = self.prob( y_left )
        
        node.right = Node()
        node.right.depth = node.depth +1
        node.right.rownum = y_right.shape[ 0 ]
        node.right.prob = self.prob( y_right )
        
        self.CreateTree( X_left,y_left,node.left )
        self.CreateTree( X_right,y_right,node.right )
        
    def fit(self,X,y):
        self.classes = np.unique( y )
        self.Tree = Node()
        self.Num = X.shape[ 0 ]
        self.Tree.prob = self.prob( y )
        self.Tree.rownum = y.shape[ 0 ]
        self.CreateTree( X,y,self.Tree )
        
    def predictSample(self,x,node):
        if node.is_terminal:
            return node.prob
        
        if x[node.feature] > node.threshlod:
            probs = self.predictSample( x,node.right )
        else:
            probs = self.predictSample( x,node.left )
            
        return probs
    
    def predict( self,X ):
        predict = [ ]
        for x in X:
            class_i = np.argmax(self.predictSample(x,self.Tree))
            predict.append( class_i )
        return np.asarray(predict) 
    
    def leaf_num(self,node):
        if node.is_terminal :
            return 1
        else:
            leaf_left = self.leaf_num( node.left )
            leaf_right = self.leaf_num( node.right )
            return leaf_left +leaf_right
        
    def R(self,node):
        if node.is_terminal:
            return min( node.prob )*node.rownum/ self.Num
        else:
            R_l = self.R(node.left)
            R_r = self.R(node.right)
            R = R_l + R_r
            return R
            
    
    def g(self,node):
        """ 
        输入节点node，计算节点的g(t)值
        """
        R_t = min( node.prob )*node.rownum/ self.Num
        R_T = self.R(node)
        N_T = self.leaf_num( node )
        return (R_t - R_T)/( N_T - 1 )
    
    def InnerNode(self,node,node_list,front,rear):
        node_list.append(node)
        rear += 1
        while rear > front:
            if node_list[ front ].left.is_terminal != True:
                node_list.append( node_list[ front ].left )
                rear += 1
            if node_list[ front ].right.is_terminal != True:
                node_list.append(node_list[ front ].right)
                rear += 1
            front += 1
    
    def getInnerNode(self,node):
        node_list = []
        front = 0
        rear = 0
        self.InnerNode( node,node_list,front,rear )
        return node_list
    
    def Prun(self,X_valid,y_valid):
        Tree_list = []
        alpha_list = []
        node_list = self.getInnerNode( self.Tree )
        square_error_of_tree=[]
        itera = len(node_list)
        while itera > 1 :
            alpha = float(np.inf)
            for node in node_list:
                g_node = self.g( node )
                alpha = min(alpha, g_node )
            node_list = node_list[::-1]
            for node in node_list:
                if self.g( node ) == alpha :
                    node.is_terminal = True
                    node_list.remove( node )
                    Tree_list.append( self.Tree )
                    alpha_list.append( alpha )
                    itera -= 1
                    break
        for subTree in Tree_list:
            self.Tree = subTree
            predict = self.predict(X_valid)
            res = np.sum( (predict - y_valid)**2 )
            square_error_of_tree.append( res )
            
        index = np.argmin( square_error_of_tree )
        self.Tree = Tree_list[ index ]
        
if __name__ == "__main__":
    data=pd.read_csv("/Users/zzk/Desktop/M_Learning/DATA/线性分类数据/Binary-classification-dataset-master/data0/data.csv", names = ['label','x1', 'x2'])
    X=data.values[:,1:]
    y=data.values[:,0]
    y_val=np.where(y==1,1,0)
    X_train, X_test, y_train, y_test = train_test_split(X, y_val, test_size=0.2, random_state=0)
    model = DecisionTreeClassifier(max_depth = 20, min_sample_leaf = 1,min_sample_split = 2)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('模型的准确率是:', accuracy_score(y_test, y_pred))
    print("**"*30)
    model.Prun(X_test,y_test)
    y_pred = model.predict(X_test)
    print('剪枝后模型的准确率是:', accuracy_score(y_test, y_pred))
    

模型的准确率是: 0.9
************************************************************
剪枝后模型的准确率是: 0.75


In [4]:
a=np.array([1,2,4,2,55,99])
b = a[::-1]
b

array([99, 55,  2,  4,  2,  1])